In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:


import numpy as np
import random
from math import sqrt



#Unitary operator to create state of 1/sqrt(3)|0> + sqrt(2)/sqrt(3)|1>
random_bit = [
    [1/sqrt(3),sqrt(2)/sqrt(3)],
    [sqrt(2)/sqrt(3),-1/sqrt(3)]
]
random_bit_transform = Operator(random_bit) 



backend = BasicSimulator()

def get_random_choice():
    #get |0> or |1> with 1/3 and 2/3 probabilty respectivley
    choice = 0
    circuit = QuantumCircuit(1,1) 
    circuit.append(random_bit_transform,[0])
    circuit.measure(range(1),range(1))
    

    qc_compiled = transpile(circuit, backend)

    job_sim = backend.run(qc_compiled, shots=1)
    result_sim = job_sim.result()

    counts = result_sim.get_counts(qc_compiled)
    observation = int(list(counts.items())[0][0])

    if observation == 1:
        #if result of first measurement is |1> then we again make a measurement with probabilites 1/2 and map the result to 1 or 2  
        circuit2 = QuantumCircuit(1,1) 
        circuit2.h(0)
        circuit2.measure(range(1),range(1))
        qc_compiled2 = transpile(circuit2, backend)

        job_sim2 = backend.run(qc_compiled2, shots=1)
        result_sim2 = job_sim2.result()

        counts2 = result_sim2.get_counts(qc_compiled2)
        observation2 = int(list(counts2.items())[0][0])
        if observation2 == 0:
            choice = 1
        else:
            choice = 2
    else:
        choice = observation
        
    return choice




2
0
2
2
1
0
0
0
0
2
0
1
1
0
0
2
2
0
2
2
1
2
2
2
1
2
0
0
2
2
0
1
0
1
2
0
1
2
0
1
2
0
2
0
2
0
1
1
0
0
1
1
1
1
0
1
0
0
2
0
1
2
1
2
1
2
1
1
1
0
2
1
0
2
0
2
1
2
2
2
1
2
2
2
1
0
2
2
0
2
2
2
1
1
1
1
1
1
1
0


In [6]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 
from math import sqrt
import random 

#Rum this cell to simulate the scenario
#Run the cell below to collect the results of running the scenario

root2 = math.sqrt(2)
denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2)


#define transform matrices corresponding to making a measurement in each of the basis in this scenario 
Z_transform_matrix = [
    [1,0],
    [0,1]
]
Z_transform = Operator(Z_transform_matrix)

X_transform_matrix = [
    [1/sqrt(2),1/sqrt(2)],
    [1/sqrt(2),-1/sqrt(2)]
]
X_transform = Operator(X_transform_matrix) 

W_transform_matrix = [ [ -1 / denom1 , (1 + root2) / denom1 ],
                        [  1 / denom2 , (root2 - 1) / denom2 ] ]
W_transform = Operator(W_transform_matrix) 


V_transform_matrix = [ [  1 / denom1 , (1 + root2) / denom1 ],
                        [ -1 / denom2 , (root2 - 1) / denom2 ] ]
V_transform = Operator(V_transform_matrix)


#define arrays to store the bases, bits and basis choices for later 
A_basis = [X_transform,W_transform,Z_transform]
B_basis = [W_transform,Z_transform,V_transform]

A_bits = []
B_bits = []

A_basis_choices = []
B_basis_choices = []

N = 100 # key length 
iterations = int(N*9/2)
backend = BasicSimulator()

basis_output_dict = {}


#loop through iterations
for i in range(iterations):
    A_i = get_random_choice()  # Generates 0, 1, or 2
    B_i = get_random_choice()
    pair = A_i,B_i

    #store the bases alice and bob chose 
    A_basis_choices.append(A_i)
    B_basis_choices.append(B_i)
    
    if pair not in basis_output_dict:
        basis_output_dict[pair] = {}
    
    #construct circuit to make engtangled btis
    circuit = QuantumCircuit(2,3) 
    circuit.h(0) 
    circuit.cx(0,1) 
    circuit.x(1)
    circuit.z(1)
    
    #Worst case eve dropper scenario
    #eve makes measurment on every one of bobs bits and stores in her own register, when this gets to bob he will measure the same bit
    circuit.measure(1,2)

    if A_basis[A_i] != Z_transform:
        circuit.unitary(A_basis[A_i],[0])
    
    if B_basis[B_i] != Z_transform:
        circuit.unitary(B_basis[B_i],[1])

    circuit.measure(range(2),range(2))
   
    
    qc_compiled = transpile(circuit, backend)
    job_sim = backend.run(qc_compiled, shots=1)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(qc_compiled)

    #get the last two bits from each measurement to measure correlation, these are the bits alice and bob recieve
    observation = list(counts.items())[0][0][-2:]

    #store alices and bobs bits they observe 
    A_bits.append(observation[1])
    B_bits.append(observation[0])

    if observation in basis_output_dict[pair]:
        basis_output_dict[pair][observation] = basis_output_dict[pair][observation] + 1
    else:
        basis_output_dict[pair][observation] = 1

print(basis_output_dict)
bXW  = basis_output_dict[(0,0)]
bXV = basis_output_dict[(0,2)]
bZW = basis_output_dict[(2,0)]
bZV = basis_output_dict[(2,2)]




{(1, 1): {'00': 27, '11': 23, '01': 3, '10': 3}, (2, 1): {'10': 17, '01': 30}, (1, 2): {'10': 14, '01': 10, '11': 6, '00': 5}, (0, 0): {'01': 11, '11': 11, '00': 13, '10': 8}, (2, 2): {'11': 19, '00': 23, '01': 3, '10': 1}, (0, 1): {'11': 13, '01': 16, '00': 13, '10': 15}, (2, 0): {'11': 24, '01': 4, '00': 24, '10': 7}, (0, 2): {'00': 16, '11': 14, '01': 15, '10': 7}, (1, 0): {'10': 24, '00': 12, '01': 10, '11': 9}}


In [7]:
def calc_E(count_dict):
    total = 0

    p00 = 0
    p11 = 0
    p01 = 0
    p10 = 0 
    
    for k in count_dict:
        total = total + count_dict[k]   

    if '00' in count_dict:
        p00 = count_dict['00']/total
    if '11' in count_dict:
        p11 = count_dict['11']/total
    if '01' in count_dict:
        p01 = count_dict['01']/total
    if '10' in count_dict:
        p10 = count_dict['10']/total
    
    return p00 + p11 -p01 - p10

#Calculate S value 
S = abs(calc_E(bXW) - calc_E(bXV) + calc_E(bZW) + calc_E(bZV))
print(S)

#Invert bobs bits
B_bits = ["0"  if b == "1" else "1" for b in B_bits]

#Construct key array based on if alice and bob chose the same basis
B_key = []
A_key = []

for i,b in enumerate(A_basis_choices):
    #they are the same when A_i = 1 ,B_i = 0 and A_i= 2,B_i = 1

    if A_basis_choices[i] == 1 and B_basis_choices[i] == 0 or A_basis_choices[i] == 2 and B_basis_choices[i] == 1:
        B_key.append(B_bits[i])
        A_key.append(A_bits[i])

print(f"S value for this simulation us {S}")
print(f"difference between S value and 2 sqrt(2) is {S-(2*sqrt(2))}")

print(f"length of key produced is {len(A_key)}")
print(f"Alice and Bob have the same key is {A_key == B_key}")

print(A_key)
print(B_key)



1.4156385165108238
S value for this simulation us 1.4156385165108238
difference between S value and 2 sqrt(2) is -1.4127886082353664
length of key produced is 102
Alice and Bob have the same key is False
['0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '1', '0', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '0', '0', '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '1', '0', '1', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0', '1', '1', '0', '1', '1', '0', '1', '0', '1', '1', '0']
['0', '1', '1', '0', '0', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '0', '1', '1', '1', '0', '1', '0', '1', '1', '1', '0', '0', '0', '1', '1', '0', '0', '0', '0', '1', '1', '0', '1', '1', '0', '1', '0', '1', '0', '0', '1', '1', '0', '0',